In [1]:
import os
from operator import itemgetter
from typing import Dict, List, Optional, Sequence

import weaviate
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from ingest_local_file import get_embeddings_model
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_models import ChatCohere
from langchain_community.vectorstores import Weaviate
from langchain_core.documents import Document
from langchain_core.language_models import LanguageModelLike
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
from pydantic import BaseModel
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import (
    ConfigurableField,
    Runnable,
    RunnableBranch,
    RunnableLambda,
    RunnablePassthrough,
    RunnableSequence,
    chain,
)

from langchain.indexes import SQLRecordManager, index

from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig

In [2]:
from langserve import RemoteRunnable

chain = RemoteRunnable("http://localhost:8080/chat/")

In [11]:
class ChatRequest(BaseModel):
    question: str
    chat_history: Optional[List[Dict[str, str]]]

chat_request = ChatRequest(question="What is streamlit?", chat_history=[{}])

In [9]:
chain.invoke(chat_request)

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

In [5]:
chatHistory = [
  {
    "human": 'Hello, AI!',
    "ai": 'Hello, human! How can I assist you today?',
  },
  {
    "human": 'What is streamlit?',
    'ai': 'Streamlit is an open-source machine learning framework for building and sharing interactive, web-based data applications [$1$]. It allows users to build applications using Python, making it accessible to a wide range of developers [$2$]. With Streamlit, you can create real-time data visualizations, run machine learning models in the browser, and add user interfaces for model predictions [$3$]. The framework also supports both local and remote development environments [$4$]',
  },
  
]

In [6]:
class ChatRequest(BaseModel):
    question: str
    chat_history: Optional[List[Dict[str, str]]]

chat_request = ChatRequest(question="Can I use it to implement model?", chat_history=chatHistory)

In [7]:
ai = ''
t = 0.6
m = 250
r = 1.0
async for chunk in chain.astream(chat_request, config=RunnableConfig(temperature=t, max_new_tokens=m, repetition_penalty=r)):
    ai += chunk
    print(chunk, end="", flush=True)

INFO:httpx:HTTP Request: POST http://localhost:8080/chat/stream "HTTP/1.1 200 OK"


 Yes, Streamlit allows you to implement models. It offers a platform for building and sharing interactive web applications that can run machine learning models in the browser [1, 3]. This means you can create real-time data visualizations, utilize your machine learning models, and even add user interfaces for model predictions [2]. The framework supports both local and remote development environments [4].

In [9]:
import google.generativeai as genai
import os 

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")    
genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [11]:
import chromadb

client = chromadb.PersistentClient(path="test")
print("Number of documents that can be inserted at once: ",client.max_batch_size)

AttributeError: 'Client' object has no attribute 'max_batch_size'